In [7]:
import glob
import os.path

import numpy as np
import matplotlib.pyplot as plt
import hvplot.xarray
import xarray as xr

from astropy.time import Time
from astropy import coordinates
from astropy.io import fits
from astropy import wcs

from dask.distributed import Client

In [8]:
from src.common import *
from src.xds_from_fits import _fits_image_to_xds

In [9]:
cl = Client()

/home/claw/.conda/envs/py39/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42981 instead
  warnings.warn(


In [47]:
fns = glob.glob('/fastpool/claw/images/first_trecs_400sqdeg_skymodel3_sub????.fits')
xds = []
for fn in fns:
    xds.append(_fits_image_to_xds(fn, chunks={'l': 1024, 'm': 1024}, verbose=True, do_sky_coords=True))

/home/claw/.conda/envs/py39/lib/python3.9/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 1.56 GiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


KeyboardInterrupt: 

In [36]:
xd0 = xds[0]
xd1 = xds[1]

In [42]:
xd0.SKY

<xarray.DataArray 'SKY' (time: 1, polarization: 1, frequency: 1, l: 40960,
                         m: 40960)> Size: 7GB
dask.array<transpose, shape=(1, 1, 1, 40960, 40960), dtype=float32, chunksize=(1, 1, 1, 40960, 40960), chunktype=numpy.ndarray>
Coordinates:
  * time          (time) int64 8B 59000
  * polarization  (polarization) <U1 4B 'I'
  * frequency     (frequency) float64 8B 1.35e+03
    velocity      (frequency) float64 8B -2.998e+14
  * l             (l) float64 328kB -4.261e-06 -8.522e-06 ... -0.1745 -0.1745
  * m             (m) float64 328kB 4.261e-06 8.522e-06 ... 0.1745 0.1745
Attributes:
    image_type:  None
    units:       None

In [41]:
xd1.SKY

<xarray.DataArray 'SKY' (time: 1, polarization: 1, frequency: 1, l: 40960,
                         m: 40960)> Size: 7GB
dask.array<transpose, shape=(1, 1, 1, 40960, 40960), dtype=float32, chunksize=(1, 1, 1, 40960, 40960), chunktype=numpy.ndarray>
Coordinates:
  * time          (time) int64 8B 59000
  * polarization  (polarization) <U1 4B 'I'
  * frequency     (frequency) float64 8B 1.35e+03
    velocity      (frequency) float64 8B -2.998e+14
  * l             (l) float64 328kB -4.261e-06 -8.522e-06 ... -0.1745 -0.1745
  * m             (m) float64 328kB 4.261e-06 8.522e-06 ... 0.1745 0.1745
Attributes:
    image_type:  None
    units:       None

In [45]:
xd0.dims

FrozenMappingWarningOnValuesAccess({'time': 1, 'polarization': 1, 'frequency': 1, 'l': 40960, 'm': 40960})

In [46]:
xr.concat([xd0, xd1], dim='l')

<xarray.Dataset> Size: 13GB
Dimensions:       (time: 1, polarization: 1, frequency: 1, l: 81920, m: 40960)
Coordinates:
  * time          (time) int64 8B 59000
  * polarization  (polarization) <U1 4B 'I'
  * frequency     (frequency) float64 8B 1.35e+03
    velocity      (frequency) float64 8B -2.998e+14
  * l             (l) float64 655kB -4.261e-06 -8.522e-06 ... -0.1745 -0.1745
  * m             (m) float64 328kB 4.261e-06 8.522e-06 ... 0.1745 0.1745
Data variables:
    SKY           (time, polarization, frequency, l, m) float32 13GB dask.array<chunksize=(1, 1, 1, 40960, 40960), meta=np.ndarray>
Attributes:
    active_mask:      None
    beam:             None
    object_name:      None
    obsdate:          {'type': 'time', 'value': 59000, 'units': 'd', 'scale':...
    observer:         me
    pointing_center:  {'value': array([ 0.14035424, -0.03101269]), 'initial':...
    description:      None

In [6]:
# create zarr
fits2zarr('/fastpool/claw/images/', '/lustre/claw/zarr2/')

In [ ]:
ds = xr.open_mfdataset(glob.glob('/lustre/claw/zarr/*'), engine='zarr', parallel=True)
npix = ds.sizes['pixel']
nside = hp.npix2nside(npix)
print(f'Dataset size: {ds.nbytes/1024**2:.1f} MB, {npix} pixels (nside={nside})')
